In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import svm

In [8]:
class regresion_lineal:
    x = []
    y = []
    beta0 = 0
    beta1 = 0
    
    def __init__(self, path, filename):
        
        #############################################
        ## Se carga la informacion en formato .npy ##
        #############################################        
        self.data_array = np.load(path+filename)
        self.df = pd.DataFrame(self.data_array)

        ###########################################################
        ## Se calcula el valor del porcentaje para entrenamiento ##
        ###########################################################
        porcentaje_80 = int(len(self.df) * .80)
                         
        #####################################################
        ## Se crean los dataset de entrenamiento y pruebas ##
        #####################################################
        self.df_entrenamiento = pd.DataFrame(self.df[:porcentaje_80])
        self.df_prueba = pd.DataFrame(self.df[porcentaje_80:])
               
        ######################################################################
        ## Se ejecuta el analisis exploratorio para el set de entrenamiento ##
        ######################################################################
        #self.analisis_exploratorio(self.df_entrenamiento)
        
        #####################################
        ## Se grafican todas las variables ##
        #####################################
        #self.graficaDataFrame(self.df_entrenamiento)

        ##############################################
        ## Se calcula el coeficiente de correlacion ##
        ##############################################
        self.comparaVariables(self.df_entrenamiento)

               
    def comparaVariables(self, df):
        s =len(df.columns)
        
        for columna in df:
            for col in df:
                x = col
                print(columna,x)
                #print(self.coeficienteCorrelacion(self.df_entrenamiento[columna],self.df_entrenamiento[subcolumna]))
            
        
    def coeficienteCorrelacion(self, dfx, dfy): #Se calculan los valores necesarios para calcular el coeficiente de correlacion
        data = {'x':dfx,'y':dfy}
        df = pd.DataFrame(data)
        df['xy'] = df['x'] * df['y'] # calculo x*y
        df['x2'] = df['x'].pow(2) # calculo cuadrados de X
        df['y2'] = df['y'].pow(2) # calculo cuadrados de y
        
        n= len(df) #calculo n
        
        xmedia = df['x'].sum() /  n #se calcula media de X
        ymedia = df['y'].sum() /  n #se calcula media de Y
        xysum = df['xy'].sum() #calculo de sumatoria de xy
        x2sum = df['x2'].sum() #calculo la sumatoria de x2
        y2sum = df['y2'].sum() #calculo la sumatoria de y2
        
        covarianza = (xysum / n) - (xmedia * ymedia)
        xmedia2 = pow(xmedia,2)
        ymedia2 = pow(ymedia,2)
        xvarianza = np.sqrt((x2sum / n) * (xmedia2))
        yvarianza = np.sqrt((y2sum / n) * (ymedia2))
        
        coeficiente = covarianza / (xvarianza * yvarianza)
        
        return coeficiente
        

    def graficaDataFrame(self,df):
        for columna in df.columns:
            plt.figure()
            sns.histplot(df[columna])
    
    def analisis_exploratorio(self,df):        
        print("Media\n",self.media(df))
        print("Maximo\n",self.numeroMaximo(df))
        print("Minimo\n",self.numeroMinimo(df))
        print("Rango\n",self.rango(df))
        print("Desviacion\n",self.desviacionEstandar(df))
            
    def media(self, df):
        return pd.DataFrame.median(df,axis=0)
    
    def numeroMaximo(self,df):
        return pd.DataFrame.max(df,axis=0)
    
    def numeroMinimo(self,df):
        return pd.DataFrame.min(df,axis=0)
    
    def rango(self,arr):
        return np.ptp(arr,axis=0)
    
    def desviacionEstandar(self,df):
        return pd.DataFrame.std(df,axis=0) 

In [9]:
path = "C:\\Users\\Marcela Ambrosio\\OneDrive\\Documentos\\Galileo\\Ciencia de Datos en python\\Proyecto Final\\proyecto_data\\"
file_name = "proyecto_training_data.npy"
pryfinal = regresion_lineal(path,file_name)

0 0
0 1
0 2
0 3
0 4
0 5
1 0
1 1
1 2
1 3
1 4
1 5
2 0
2 1
2 2
2 3
2 4
2 5
3 0
3 1
3 2
3 3
3 4
3 5
4 0
4 1
4 2
4 3
4 4
4 5
5 0
5 1
5 2
5 3
5 4
5 5
